# Predicting Moral Values in Text
### This Code offers predicting moral values from the MoralBERT weights deployad in Hugging Face.

In [6]:
# Libraries:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from huggingface_hub import PyTorchModelHubMixin
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F

In [7]:
# BERT model and tokenizer:
bert_model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [10]:
class MyModel(
    nn.Module,
    PyTorchModelHubMixin,
    # optionally, you can add metadata which gets pushed to the model card
    # repo_url="your-repo-url",
    pipeline_tag="text-classification",
    license="mit",
):
    def __init__(self, bert_model, moral_label=2):

        super(MyModel, self).__init__()
        self.bert = bert_model
        bert_dim = 768
        self.invariant_trans = nn.Linear(768, 768)
        self.moral_classification = nn.Sequential(nn.Linear(768,768),
                                                      nn.ReLU(),
                                                      nn.Linear(768, moral_label))

    def forward(self, input_ids, token_type_ids, attention_mask):
        pooled_output = self.bert(input_ids,
                                token_type_ids = token_type_ids,
                                attention_mask = attention_mask).last_hidden_state[:,0,:]


        pooled_output = self.invariant_trans(pooled_output)


        logits = self.moral_classification(pooled_output)

        return logits

In [11]:
def preprocessing(input_text, tokenizer):
    '''
    Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
    '''
    return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 150,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_token_type_ids = True,  # Add this line
                        return_tensors = 'pt',
                        truncation=True
                   )

In [12]:
# Example list of sentences
sentences = [
    "This is good, but I do not understand it!",
    "I care a lot for my health and well-being",
    "You have btrayed your country!!",
    "Breaking the law is bad."
    # Add more sentences as needed
]

# the list of Moral (MFT) values
mft_values = ["care", "harm", "fairness", "cheating", "loyalty", "betrayal",
              "authority", "subversion", "purity", "degradation"]


# function to load the model, predict the score, and return the second value
def get_model_score(sentence, mft):
    repo_name = f"vjosap/moralBERT-predict-{mft}-in-text"

    # loading the model
    model = MyModel.from_pretrained(repo_name, bert_model=bert_model)

    # preprocessing the text
    encodeds = preprocessing(sentence, tokenizer)

    # predicting the mft score
    output = model(**encodeds)
    score = F.softmax(output, dim=1)

    # extracting and return the second value from the tensor
    mft_value = score[0, 1].item()

    return mft_value

# initialising a list to accumulate the results
results = []

# sequential execution of predictions
for sentence in sentences:
    # dictionary to store scores for the current sentence
    sentence_scores = {"sentence": sentence}

    # iterate through each MFT model and get the score
    for mft in mft_values:
        sentence_scores[mft] = get_model_score(sentence, mft)

    results.append(sentence_scores)

results_df = pd.DataFrame(results)

# display the final results
results_df

# save the DataFrame to a CSV file
# results_df.to_csv("moral_foundation_scores.csv", index=False)


,sentence,care,harm,fairness,cheating,loyalty,betrayal,authority,subversion,purity,degradation
0,"This is good, but I do not understand it!",0.017906,0.011826,0.000711,0.000608,0.001132,0.000901,0.000606,0.002593,0.001033,0.007981
1,I care a lot for my health and well-being,0.913170,0.000821,0.002522,0.000750,0.001102,0.010791,0.001195,0.001110,0.002080,0.008218
2,You have btrayed your country!!,0.091193,0.001212,0.003562,0.423068,0.992904,0.770573,0.002085,0.003547,0.000779,0.008177
3,Breaking the law is bad.,0.019675,0.083457,0.000525,0.002635,0.000824,0.003747,0.925532,0.990706,0.001234,0.025948
